## Optimization exercise

**Goal: Train the 2nd order polynomial predictor using gradient descent. Optimize the stepsizes and compare against scikit-learn's implementation.**

In [3]:
# IMPORT PACKAGES
import pandas as pd
import numpy as np


In [8]:
data_path = 'data/'
nb = pd.read_csv(data_path+'nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']]
y = nb['PTS']

#### Task I
Download the data from [here](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing&resourcekey=0-16O9Fc5eaJH99-M7AHqHOg).


#### TASK II
Create a function `psi(x)`, which transforms features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features (add each feature squared and each pair of features multiplied with every other)

**Input:** DataFrame x from above. It contains columns AST, REB, STL

**Output:** DataFrame with columns: AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB. The number of rows should be the same as Input.

In [9]:
def psi(x):
    ret = pd.DataFrame(x)
    ret['1'] = 1
    ret['AST^2'] = ret['AST'] ** 2
    ret['REB^2'] = ret['REB'] ** 2
    ret['STL^2'] = ret['STL'] ** 2
    ret['ASTSTL'] = ret['AST'] * ret['STL']
    ret['REBSTL'] = ret['REB'] * ret['STL']
    ret['ASTREB'] = ret['AST'] * ret['REB']

    return ret

In [11]:
poly_nb = psi(x)
poly_nb

,AST,REB,STL,1,AST^2,REB^2,STL^2,ASTSTL,REBSTL,ASTREB
0,41,43,14,1,1681,1849,196,574,602,1763
1,23,43,8,1,529,1849,64,184,344,989
2,20,39,7,1,400,1521,49,140,273,780
3,19,47,6,1,361,2209,36,114,282,893
4,21,43,4,1,441,1849,16,84,172,903
...,...,...,...,...,...,...,...,...,...,...
7375,17,39,10,1,289,1521,100,170,390,663
7376,26,40,10,1,676,1600,100,260,400,1040
7377,23,52,8,1,529,2704,64,184,416,1196
7378,23,41,11,1,529,1681,121,253,451,943


#### Task III
Create a transformed data matrix **X**, where each **x** is mapped to psi(x).

HINT: We need to apply our function from Task II to matrix (DataFrame) x

#### Task IV
Create a function `p2(x,w)`, which outputs the value of the polynomial ata given row of `x` for given parameters `w`.

Inputs:
- x: DataFrame from above
- w: vector which represents beta coeficients for each column of X from the Task III.

Ouputs:
- Series of the same length as DataFrame x. Each value is a dot product of particular row in DataFrame and coeficients `w`.


HINT:
- length of w needs to be the same as number of columns of the dataframe that is returned from function `psi(x)`

Example Input:

`p2(x, [0.06, 0.05,0.03,0.01,0.02,0.02,0.04, 0.03,0.02,0.01])`

Example Output:

```
0       130.37
1        76.19
2        61.21
3        74.51
4        64.97
         ...  
7375     63.01
7376     79.59
7377     97.25
7378     78.85
7379     61.53
Length: 7380, dtype: float64

```

Our columns in the DataFrame **X** that is the output of `psi(x)` were in this order: `AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB`. If your columns are in different order the result can be different even for the **w**.

In [32]:
coef =  [0.06, 0.05,0.03,0.01,0.02,0.02,0.04, 0.03,0.02,0.01]

def p2_help(w,x):
    sum = 0
    for i in range(0,len(w)):
        sum += x[i] * w[i]
    return sum

def p2(x,w):
    return x.apply(lambda z: p2_help(w,z),axis=1)


In [33]:
y_preds = p2(x,coef)
print(y_preds)

0       130.37
1        76.19
2        61.21
3        74.51
4        64.97
         ...  
7375     63.01
7376     79.59
7377     97.25
7378     78.85
7379     61.53
Length: 7380, dtype: float64


#### Task V
Create a function `Loss(x,y,w)`, which computes the squared loss of predicting **y** from **x** by `p2(x,w)` using parameters **w**. We have specified **y** as the variable PTS above. We will predict scored points (PTS) based on assists, rebounds and steals.


HINTS: 
- Output of `p2(x,w)` represents our predictions. `y_pred = p2(x,w)`
- Loss can be computed as:

```
np.mean((y_pred - y)**2)
```

In [34]:
def Loss(x,y,w):
    y_pre = p2(x,w)
    return np.mean((y_pre - y)**2)



In [35]:
print(Loss(x,y,coef))

891.9364108807605


#### Task VI
Code up the gradient descent. It should input **x**, target variable **y**, stepsize **alpha**, coeficients **w** and **maximum number of iterations**.

Steps:
1. transform input `x`
2. compute initial loss for given, x,y and w and append it to the empty list.
3. Inside the for loop, update each element of **w**, **w[i]**, using gradient descent.

HINT: `w[i] = w[i] - alpha * (1.0 / m) * errors_i.sum()` where `errors_i = (y_pred - y) * X.iloc[:, i]`. We are scaling the errors by multiplicating with values that are relevant for coeficients `w[i]` (column `i` of DataFrame `X`, output of `psi(x)`).

4. compute new loss using updated `w` and append to the list that we created in the step 2.
5. repeat steps 3 and 4 for max number of iterations times.

In [98]:
def gradient_descent(x,y,w,alpha,max_iterations=20):
    poly_x = psi(x)
    y_pre = p2(x,w)
    loss = np.mean((y_pre - y)**2)


    loss_df = pd.DataFrame()
    loss_df[0] = loss

    the_w = pd.DataFrame(w)
    print(the_w[0])
    for j in range(1,max_iterations):
        for i in range(1,len(w)):
            errors_i = (y_pre - y) * x.iloc[:,i]
            the_w.loc[i] = the_w.loc[i-1]
            the_w.loc[i] = the_w.loc[i] - alpha * (1.0 / max_iterations) * errors_i.sum()
    # print(the_w[i])
        loss_df.loc[j] = Loss(x,y,the_w[i])

    return loss_df


In [99]:
gradient_descent(x,y,coef,.25,10)

0    6.000000e-02
1    9.143067e+04
2    1.079590e+05
3    1.102417e+05
4    1.222909e+06
5    4.942526e+06
6    5.079680e+06
7    5.437476e+06
8    6.090103e+06
9    8.056459e+06
Name: 0, dtype: float64


KeyError: 9

#### Task VII
Choose an arbitrary **w** and **alpha** and run gradient descent for 100 iterations. How does the loss behave? Does it converge to something?


#### Task VIII
Can you find which **alpha** from `[1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]` has the smallest loss after 100 iterations for a given **w**?